In [ ]:
import tensorflow as tf
import keras
from keras import layers, models
from keras import losses, optimizers, metrics
from keras.preprocessing.image import ImageDataGenerator

import numpy as np

In [ ]:
import os
import pickle

In [ ]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip"
dataset = keras.utils.get_file("traffic-signs-data.zip",
                               url,
                               extract=True)

print(f"File path:{dataset}")

123524425/123524425 [==============================] - 1s 0us/step
File path:/root/.keras/datasets/traffic-signs-data.zip


In [ ]:
dataset_dir = os.path.dirname(dataset)

In [ ]:
os.listdir(dataset_dir)

['test.p', 'valid.p', 'train.p', 'traffic-signs-data.zip']

In [ ]:
train_data = pickle.load(open(os.path.join(dataset_dir, "train.p"), "rb"))
val_data = pickle.load(open(os.path.join(dataset_dir, "valid.p"), "rb"))
test_data = pickle.load(open(os.path.join(dataset_dir, "test.p"), "rb"))

In [ ]:
train_data.keys()

dict_keys(['coords', 'labels', 'features', 'sizes'])

In [ ]:
train_features, train_labels = train_data['features'], train_data['labels']
val_features, val_labels = val_data['features'], val_data['labels']
test_features, test_labels = test_data['features'], test_data['labels']

In [ ]:
train_labels.shape

(34799,)

In [ ]:
print(f"Number of training examples: {len(train_features)}")
print(f"Number of training labels: {len(train_labels)}")
print(f"Number of validation examples: {len(val_features)}")
print(f"Number of validtion labels: {len(val_labels)}")
print(f"Number of test examples: {len(test_features)}")
print(f"Number of test labels: {len(test_labels)}")


Number of training examples: 34799
Number of training labels: 34799
Number of validation examples: 4410
Number of validtion labels: 4410
Number of test examples: 12630
Number of test labels: 12630


In [ ]:
num_train_samples, *input_shape = train_features.shape
num_classes = 43
print(f"Input shape:{input_shape}")

Input shape:[32, 32, 3]


In [ ]:
print(f"Pixel Range:{np.min(train_features), np.max(train_features)}")

Pixel Range:(0, 255)


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).shuffle(buffer_size=num_train_samples)
val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).shuffle(buffer_size=val_features.shape[0])
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels))

In [ ]:
for i, l in train_ds.take(1):
  print(l)
  print(tf.one_hot(l, depth=10))

tf.Tensor(4, shape=(), dtype=uint8)
tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32)


In [ ]:
# Method 1
from keras.preprocessing.image import ImageDataGenerator
augmentator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)


In [ ]:
BATCH_SIZE = 128

train_ds = train_ds.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Augmented image data

ds_train_aug = augmentator.flow((train_features, train_labels), batch_size=BATCH_SIZE)

### Model Architecture

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.layers import Rescaling

#### No data augmentation

In [ ]:
model = Sequential([
    Rescaling(scale=1./255, input_shape=input_shape),
    Conv2D(32, (2, 2), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(32, (2, 2), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (2, 2), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (2, 2), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(512, activation="relu"),
    Dense(num_classes, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        416       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        4128      
                                                                 
 batch_normalization_1 (Bat  (None, 16, 16, 32)        1

In [ ]:
model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer=optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=50,
          callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join('.', model.name)+'.h5',
                                                        monitor='val_accuracy',
                                                        verbose=1,
                                                        save_best_only=True
                                                        )])

Epoch 1/50
271/272 [============================>.] - ETA: 0s - loss: 0.0101 - accuracy: 0.9969
Epoch 1: val_accuracy improved from -inf to 0.90658, saving model to ./sequential.h5
272/272 [==============================] - 4s 6ms/step - loss: 0.0100 - accuracy: 0.9970 - val_loss: 0.7219 - val_accuracy: 0.9066
Epoch 2/50
270/272 [============================>.] - ETA: 0s - loss: 0.0089 - accuracy: 0.9972
Epoch 2: val_accuracy did not improve from 0.90658
272/272 [==============================] - 1s 5ms/step - loss: 0.0088 - accuracy: 0.9972 - val_loss: 0.8768 - val_accuracy: 0.8955
Epoch 3/50
265/272 [============================>.] - ETA: 0s - loss: 0.0076 - accuracy: 0.9976
Epoch 3: val_accuracy did not improve from 0.90658
272/272 [==============================] - 1s 5ms/step - loss: 0.0074 - accuracy: 0.9976 - val_loss: 0.7859 - val_accuracy: 0.8959
Epoch 4/50
262/272 [===========================>..] - ETA: 0s - loss: 0.0017 - accuracy: 0.9994
Epoch 4: val_accuracy improved from 

In [ ]:
!zip -r classify_traffic_signs.zip sequential.h5

  adding: sequential.h5 (deflated 11%)


#### Augmented Data

In [ ]:
model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer=optimizers.RMSprop(),
              metrics=["accuracy"])
model.fit(ds_train_aug,
          validation_data=(val_features, val_labels),
          epochs=30)

Epoch 1/30
272/272 [==============================] - 19s 64ms/step - loss: 0.7264 - accuracy: 0.8202 - val_loss: 0.6867 - val_accuracy: 0.8728
Epoch 2/30
272/272 [==============================] - 17s 62ms/step - loss: 0.2904 - accuracy: 0.9013 - val_loss: 0.7952 - val_accuracy: 0.8574
Epoch 3/30
272/272 [==============================] - 17s 64ms/step - loss: 0.2257 - accuracy: 0.9227 - val_loss: 0.6866 - val_accuracy: 0.8662
Epoch 4/30
272/272 [==============================] - 17s 63ms/step - loss: 0.1972 - accuracy: 0.9324 - val_loss: 0.7950 - val_accuracy: 0.8571
Epoch 5/30
272/272 [==============================] - 18s 64ms/step - loss: 0.1629 - accuracy: 0.9447 - val_loss: 0.9114 - val_accuracy: 0.8517
Epoch 6/30
272/272 [==============================] - 17s 63ms/step - loss: 0.1500 - accuracy: 0.9487 - val_loss: 0.8232 - val_accuracy: 0.8476
Epoch 7/30
272/272 [==============================] - 17s 63ms/step - loss: 0.1469 - accuracy: 0.9498 - val_loss: 0.8201 - val_accuracy: